In [1]:
# GPU utils, can be omitted
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="6"

### Load model

In [7]:
import sys
sys.path.append('../')

from model_utils import load_model, DATA_ROOT, SAVE_ROOT, get_layer_names

MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'

model = load_model(MODEL_NAME)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [8]:
layer_names, conv_layer_mask = get_layer_names(model)
fc_layer_mask = (1 - conv_layer_mask).astype(bool)

print(layer_names[fc_layer_mask])

['classifier.0' 'classifier.3' 'classifier.6']


### Create data loaders

In [9]:
# Choose from 'mnist', 'cifar10', 'imagenet'
DATASET = 'imagenet'
NUM_WORKERS = 4

##### Load data
import dataloaders

bs = 32

loaders = dataloaders.get_loader(bs, DATASET, DATA_ROOT, num_workers = NUM_WORKERS, simple_normalize=False)

Building imagenet data loader with 4 workers


# Iterative fine-tunning

- Compressed_model = Initial_model
- For i in n_iters:
    - Select layers to compress (select layer names from Initial_model, compress same-name layers from Compressed_model)
    - Select compression ranks,
        - For Tucker:
            - Set rank = None and vbmf_weaken_factor != None for VBMF rank selection (Tucker2 only).
            - Set rank = scalar for rank selection based on (scalar x) layer parameter reduction.
            - Set rank = (rank_cout, rank_cin), elswhere
        - For CP:
            - Set rank = scalar (absolute rank value).
    - Compress layers in Compressed_model
    - Fine-tune Compressed_model
      - Save current_chekpoint and best_model
      - Compressed_model = best_model

In [10]:
from model_utils import get_layer_names
from tensor_compression import get_compressed_model, finetune

import copy
import torch
import os
import numpy as np

def split_resnet_layers_by_blocks(lnames):
    starts = ['conv1'] + ['layer{}'.format(i) for i in range(1,5)]

    start_idx = 0
    blocks_idxs = []
    layer_names_by_blocks = []

    for s in starts:
        curr_block =  [l for l in lnames if l.startswith(s)]
        layer_names_by_blocks.append(curr_block)

        blocks_idxs.append(np.arange(start_idx, start_idx+len(curr_block)))
        start_idx += len(curr_block)

    return blocks_idxs

### Scenario: at each iteration  fine-tune  compressed model after  selected layers are compressed

In [11]:
GLOBAL_COMPRESS_ITER_NUM = 2
FT_EPOCHS = 1

layer_names, conv_layer_mask = get_layer_names(model)
fc_layer_mask = (1 - conv_layer_mask).astype(bool)

CONV_SPLIT = 3
FC_SPLIT = 1
n_layers = len(layer_names)

# decomposition_conv = 'cp4'
# decomposition_conv = 'cp3'
decomposition_conv = 'tucker2'
decomposition_fc = 'svd'

# RANK_SELECTION = 'vbmf'
RANK_SELECTION = 'nx'
# RANK_SELECTION = 'custom'

if RANK_SELECTION == 'vbmf':
    WEAKEN_FACTOR = 0.9
    X_FACTOR = 0
    rank_selection_suffix = "/wf:{}".format(WEAKEN_FACTOR)
elif RANK_SELECTION == 'nx':
    WEAKEN_FACTOR = None  
    X_FACTOR = 5
    rank_selection_suffix = "/{}x".format(X_FACTOR)
    
    
    
if MODEL_NAME == 'vgg16_imagenet':
    ranks_conv = [None] + [-X_FACTOR]*(len(layer_names[conv_layer_mask])-1)
    ranks_fc = [-X_FACTOR]*(len(layer_names[fc_layer_mask]))
elif MODEL_NAME == 'resnet50_imagenet':
    ranks_conv = [None if not name.endswith('conv2') else -X_FACTOR
                  for name in layer_names[conv_layer_mask]]
    ranks_fc = [-X_FACTOR]*(len(layer_names[fc_layer_mask]))
        

ranks = np.array([None]*len(layer_names))
ranks[conv_layer_mask] = ranks_conv
ranks[fc_layer_mask] = ranks_fc

decompositions = np.array([None]*len(layer_names))
decompositions[conv_layer_mask] = decomposition_conv
decompositions[fc_layer_mask] = decomposition_fc
        
        
SPLIT_FACTOR = CONV_SPLIT + FC_SPLIT
save_dir = "{}/models_finetuned/{}/{}/rank_selection:{}{}/layer_groups:{}".format(SAVE_ROOT,
                                                                                   MODEL_NAME,
                                                                                   decomposition_conv + '-' + decomposition_fc,
                                                                                   RANK_SELECTION,
                                                                                   rank_selection_suffix, 
                                                                                   SPLIT_FACTOR)
loggs_dir = "{}/loggs".format(save_dir)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(loggs_dir):
    os.makedirs(loggs_dir)

device = 'cuda'
train_iters = 1
val_iters = 1
 
RESNET_SPLIT = True
if MODEL_NAME == 'resnet50_imagenet' and RESNET_SPLIT:
    split_tuples = split_resnet_layers_by_blocks(layer_names[conv_layer_mask])[::-1]
else:
    split_tuples = np.array_split(np.arange(n_layers)[conv_layer_mask], CONV_SPLIT)[::-1]
split_tuples.append(np.array_split(np.arange(n_layers)[fc_layer_mask], FC_SPLIT))

In [12]:
compressed_model = copy.deepcopy(model)
for global_iter in range(GLOBAL_COMPRESS_ITER_NUM):
                                                                                
    for local_iter, tupl in enumerate(split_tuples):
        lname, rank, decomposition = layer_names[tupl], ranks[tupl], decompositions[tupl]        
        
        if isinstance(tupl[0], np.ndarray):
            print(lname, tupl[0])
            print('yohoo')


#         compressed_model = get_compressed_model(compressed_model,
#                                        ranks=rank, layer_names = lname,
#                                         decompositions = decomposition,
#                                         vbmf_weaken_factor = WEAKEN_FACTOR)
        
#         suffix = "_iter:{}-{}".format(global_iter, local_iter)

        
#         finetune.fine_tune(compressed_model, loaders, device, save_dir,
#                            train_iters = train_iters, val_iters = val_iters, ft_epochs = FT_EPOCHS,
#                            suffix = suffix, loggs_dir = loggs_dir)
#         try:
#             compressed_model = torch.load('{}/model_best{}.pth.tar'.format(save_dir, suffix))
#         except:
#             print('Stop compression')
#             break

['classifier.0' 'classifier.3' 'classifier.6'] [13 14 15]
yohoo
['classifier.0' 'classifier.3' 'classifier.6'] [13 14 15]
yohoo


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """


In [18]:
compressed_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

# Count parameters

In [9]:
from collections import defaultdict, Counter
def count_params(model):
    n_params = defaultdict()
    
    for name, param in model.named_parameters():
        n_params[name] = param.numel()
    return n_params

In [10]:
def count_params_by_layers(params_count_dict):
    params_count_dict_modif = defaultdict()
    
    for k,v in params_count_dict.items():
        if '-' not in  k:
            k_head  = k.strip('.weight').strip('.bias')
            try:
                params_count_dict_modif[k_head] += params_count_dict[k]
            except:
                params_count_dict_modif[k_head] = params_count_dict[k]
        else:
            k_head = '.'.join(k.split('-')[0].split('.')[:-1])
            try:
                params_count_dict_modif[k_head] += params_count_dict[k]
            except:
                params_count_dict_modif[k_head] = params_count_dict[k]

    return params_count_dict_modif

In [11]:
params_count_dict_m = count_params(model)
params_count_dict_cm = count_params(compressed_model)

sum(params_count_dict_m.values())/sum(params_count_dict_cm.values())

1.952920589581533

In [12]:
np.array(list((count_params_by_layers(params_count_dict_m).values()))) / np.array(list((count_params_by_layers(params_count_dict_cm).values())))

array([ 1.        ,  1.        ,  1.        ,  1.        ,  5.13998885,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  5.13998885,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  5.13998885,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        , 25.14168798,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        , 25.14168798,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        , 25.14168798,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        , 25.14168798,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
       24.05383141,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        , 24.05383141,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        , 24.05383141,
        1.        ,  1.        ,  1.        ,  1.        ,  1.  